In [1]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [51]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

import time

In [15]:
SEED = 7777

In [3]:
DATA_PATH = '/content/gdrive/MyDrive/KaggleTabularPlaygroundSeries/data'
df_train = pd.read_csv(f'{DATA_PATH}/train.csv')

In [4]:
df_train.shape

(200000, 288)

In [5]:
df_train.head(2)

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,A0T0G10C0,A0T1G0C9,A0T1G1C8,A0T1G2C7,A0T1G3C6,A0T1G4C5,A0T1G5C4,A0T1G6C3,A0T1G7C2,A0T1G8C1,A0T1G9C0,A0T2G0C8,A0T2G1C7,A0T2G2C6,A0T2G3C5,A0T2G4C4,A0T2G5C3,A0T2G6C2,A0T2G7C1,A0T2G8C0,A0T3G0C7,A0T3G1C6,A0T3G2C5,A0T3G3C4,A0T3G4C3,A0T3G5C2,A0T3G6C1,A0T3G7C0,A0T4G0C6,...,A5T3G2C0,A5T4G0C1,A5T4G1C0,A5T5G0C0,A6T0G0C4,A6T0G1C3,A6T0G2C2,A6T0G3C1,A6T0G4C0,A6T1G0C3,A6T1G1C2,A6T1G2C1,A6T1G3C0,A6T2G0C2,A6T2G1C1,A6T2G2C0,A6T3G0C1,A6T3G1C0,A6T4G0C0,A7T0G0C3,A7T0G1C2,A7T0G2C1,A7T0G3C0,A7T1G0C2,A7T1G1C1,A7T1G2C0,A7T2G0C1,A7T2G1C0,A7T3G0C0,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,0,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.0002,-0.00024,-0.0002,-0.000114,-0.000043,-0.00001,-9.536743e-07,-0.00001,-0.000086,-0.000343,-0.000801,-0.001202,-0.001202,-0.000801,-0.000343,-0.000086,-0.00001,-0.000043,-0.000343,-0.001202,-0.002403,-0.003004,-0.002403,-0.001202,-0.000343,-0.000043,-0.000114,-0.000801,-0.002403,-0.004005,-0.004005,-0.002403,-0.000801,-0.000114,-0.0002,...,-0.002403,0.008798,-0.001202,0.00976,-0.0002,-0.000801,-0.001202,-0.000801,-0.0002,-0.000801,-0.002403,-0.002403,-0.000801,-0.001202,-0.002403,-0.001202,-0.000801,-0.000801,-0.0002,-0.000114,-0.000343,-0.000343,-0.000114,-0.000343,-0.000687,-0.000343,-0.000343,-0.000343,-0.000114,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,Streptococcus_pyogenes
1,1,-9.536743e-07,-0.00001,-0.000043,0.000886,-0.0002,0.00076,-0.0002,-0.000114,-0.000043,-0.00001,-9.536743e-07,-0.00001,-0.000086,-0.000343,0.000199,-0.000202,0.001798,-0.000801,-0.000343,-0.000086,-0.00001,-0.000043,-0.000343,0.001798,-0.000403,0.001996,0.003597,-0.001202,-0.000343,-0.000043,-0.000114,-0.000801,-0.000403,0.002995,0.002995,0.000597,0.001199,-0.000114,-0.0002,...,0.000597,0.002798,0.001798,0.00076,-0.0002,-0.000801,-0.000202,-0.000801,0.0008,0.000199,0.003597,0.001597,-0.000801,-0.000202,-0.001403,0.000798,0.001199,0.001199,0.0008,-0.000114,-0.000343,-0.000343,-0.000114,-0.000343,0.001313,0.000657,0.001657,0.001657,0.000886,-0.000043,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-0.00001,-0.00001,-0.00001,-9.536743e-07,Salmonella_enterica


In [6]:
df_train.describe()

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,A0T0G10C0,A0T1G0C9,A0T1G1C8,A0T1G2C7,A0T1G3C6,A0T1G4C5,A0T1G5C4,A0T1G6C3,A0T1G7C2,A0T1G8C1,A0T1G9C0,A0T2G0C8,A0T2G1C7,A0T2G2C6,A0T2G3C5,A0T2G4C4,A0T2G5C3,A0T2G6C2,A0T2G7C1,A0T2G8C0,A0T3G0C7,A0T3G1C6,A0T3G2C5,A0T3G3C4,A0T3G4C3,A0T3G5C2,A0T3G6C1,A0T3G7C0,A0T4G0C6,...,A5T3G1C1,A5T3G2C0,A5T4G0C1,A5T4G1C0,A5T5G0C0,A6T0G0C4,A6T0G1C3,A6T0G2C2,A6T0G3C1,A6T0G4C0,A6T1G0C3,A6T1G1C2,A6T1G2C1,A6T1G3C0,A6T2G0C2,A6T2G1C1,A6T2G2C0,A6T3G0C1,A6T3G1C0,A6T4G0C0,A7T0G0C3,A7T0G1C2,A7T0G2C1,A7T0G3C0,A7T1G0C2,A7T1G1C1,A7T1G2C0,A7T2G0C1,A7T2G1C0,A7T3G0C0,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
count,200000.000000,2.000000e+05,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,2.000000e+05,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,2.000000e+05,2.000000e+05,2.000000e+05
mean,99999.500000,6.421457e-07,-0.000003,-0.000014,-0.000010,0.000005,0.000025,0.000014,-0.000009,-0.000028,-0.000008,-7.498743e-07,0.000021,-0.000055,-0.000162,-0.000185,-0.000141,0.000142,-0.000092,-0.000100,-0.000053,-0.000008,-0.000026,-0.000216,-0.000462,-0.000554,-0.000265,-0.000296,-0.000302,-0.000150,-0.000027,-0.000055,-0.000418,-0.000825,-0.000761,-0.000704,-0.000600,-0.000196,-0.000056,-0.000074,...,0.004902,0.002171,0.004145,0.002231,0.003025,0.000045,0.000240,0.000586,0.000464,0.000210,0.000458,0.001835,0.001989,0.000822,0.001292,0.002505,0.001791,0.001693,0.001998,0.000853,0.000116,0.000445,0.000577,0.000287,0.000647,0.001469,0.001173,0.001035,0.001394,0.000671,0.000135,0.000348,0.000253,0.000464,0.000581,0.000379,0.000062,8.982134e-05,1.170014e-04,2.215407e-07
std,57735.171256,8.654927e-05,0.000132,0.000287,0.000436,0.000683,0.000869,0.000775,0.000441,0.000107,0.000083,5.140499e-06,0.000165,0.000116,0.000665,0.001375,0.002061,0.002193,0.001524,0.000760,0.000157,0.000013,0.000093,0.000469,0.001172,0.002358,0.003221,0.002652,0.001545,0.000688,0.000201,0.000303,0.000824,0.001848,0.003175,0.003130,0.002097,0.000990,0.000343,0.000507,...,0.007141,0.004343,0.005278,0.003982,0.004298,0.000774,0.001676,0.002131,0.001883,0.001017,0.001858,0.003521,0.003852,0.002275,0.002977,0.004863,0.003363,0.003309,0.003535,0.002165,0.000780,0.001517,0.001618,0.001102,0.001814,0.002874,0.002159,0.002281,0.002648,0.001711,0.000711,0.001117,0.000914,0.001238,0.001528,0.001182,0.000466,5.475606e-04,6.788277e-04,2.050081e-05
min,0.000000,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,-9.536743e-07,-0.000010,-0.000086,-0.000343,-0.000801,-0.001202,-0.001202,-0.000801,-0.000343,-0.000086,-0.000010,-0.000043,-0.000343,-0.001202,-0.002403,-0.003004,-0.002403,-0.001202,-0.000343,-0.000043,-0.000114,-0.000801,-0.002403,-0.004005,-0.004005,-0.002403,-0.000801,-0.000114,-0.000200,...,-0.004807,-0.002403,-0.001202,-0.001202,-0.000240,-0.000200,-0.000801,-0.001202,-0.000801,-0.000200,-0.000801,-0.002403,-0.002403,-0.000801,-0.001202,-0.002403,-0.001202,-0.000801,-0.000801,-0.000

In [7]:
target_names = list(set(df_train['target']))
df_targets = {}
for name in target_names:
    df_targets[name] = df_train[df_train['target'] == name].reset_index(drop=True)

In [8]:
for name in target_names:
    print(name, len(df_targets[name]))

Escherichia_coli 19958
Campylobacter_jejuni 20063
Bacteroides_fragilis 20139
Enterococcus_hirae 19947
Staphylococcus_aureus 19929
Salmonella_enterica 20030
Streptococcus_pneumoniae 20074
Klebsiella_pneumoniae 19847
Streptococcus_pyogenes 20076
Escherichia_fergusonii 19937


In [12]:
features = df_train.columns[1:-1]
print(features)

Index(['A0T0G0C10', 'A0T0G1C9', 'A0T0G2C8', 'A0T0G3C7', 'A0T0G4C6', 'A0T0G5C5',
       'A0T0G6C4', 'A0T0G7C3', 'A0T0G8C2', 'A0T0G9C1',
       ...
       'A8T0G0C2', 'A8T0G1C1', 'A8T0G2C0', 'A8T1G0C1', 'A8T1G1C0', 'A8T2G0C0',
       'A9T0G0C1', 'A9T0G1C0', 'A9T1G0C0', 'A10T0G0C0'],
      dtype='object', length=286)


In [42]:
label_encoder = preprocessing.LabelEncoder()
df_train['target'] = label_encoder.fit_transform(df_train['target'])

In [45]:
list(label_encoder.classes_)

['Bacteroides_fragilis',
 'Campylobacter_jejuni',
 'Enterococcus_hirae',
 'Escherichia_coli',
 'Escherichia_fergusonii',
 'Klebsiella_pneumoniae',
 'Salmonella_enterica',
 'Staphylococcus_aureus',
 'Streptococcus_pneumoniae',
 'Streptococcus_pyogenes']

In [52]:
X_train = df_train[features]
y_train = df_train['target']
rfclf = RandomForestClassifier(n_estimators=300, n_jobs=-1)

start_time = time.time()
rfclf.fit(X_train, y_train)
print(f'Train Time Elapsed: {time.time() - start_time}')

Train Time Elapsed: 311.6787459850311


In [54]:
df_test = pd.read_csv(f'{DATA_PATH}/test.csv')
print(len(df_test))

100000


In [56]:
X_test = df_test[features]

In [57]:
y_test = rfclf.predict(X_test)

In [58]:
df_submit = pd.read_csv(f'{DATA_PATH}/sample_submission.csv')
df_submit.head()

,row_id,target
0,200000,Streptococcus_pneumoniae
1,200001,Streptococcus_pneumoniae
2,200002,Streptococcus_pneumoniae
3,200003,Streptococcus_pneumoniae
4,200004,Streptococcus_pneumoniae


In [59]:
y_name = label_encoder.inverse_transform(y_test)

In [61]:
df_submit['target'] = y_name

In [63]:
df_submit.to_csv(f'{DATA_PATH}/0202_rfclf_baseline.csv', index=False)